In [1]:
%%html
<style>
.output_wrapper, .output { height:auto !important; max-height:10000px; } /* your desired max-height here */
.output_scroll {     box-shadow:none !important; webkit-box-shadow:none !important; }
.noborder, .noborder tr, .noborder th, .noborder td { border-collapse: collapse; border: 0px; border: none!important; }
</style>

In [2]:
import sys, os
sys.path.append('./modules')
from Monitoring_Tools import *
from OpenStack_Tools  import *

In [3]:
platforms = ['demo8', 'poc1', 'poc2', 'nfv5']
#platforms = ['demo8', 'poc1' ]

HTML_OPS={}
STATUSES={}
DISK_USAGE={}

thresholds=[70,90]
colours=['lightgreen','orange','red']
for platform in platforms:
    inventory, HTML_OP, PING_STATUS, PING_PORTS_STATUS, ENDPOINTS_STATUS, VMS_STATUS, \
    HIGHEST_DISK_PC, HIGHEST_DISK_PC_HOST, SUMMARY_HIGHEST_DISK_HTML, HIGHEST_DISK_HTML = platformStatus(platform, disk_thresholds=[70,90])
    HTML_OPS[platform] = HTML_OP + HIGHEST_DISK_HTML
    STATUSES[platform] = [ SUMMARY_HIGHEST_DISK_HTML, PING_STATUS, PING_PORTS_STATUS, ENDPOINTS_STATUS, VMS_STATUS ]


-------- Checking platform <demo8>
2 flavors, 8 images, 0 servers
ping(demo8co0[10.3.162.143:22]) ... ping(demo8co1[10.3.162.141:22]) ... ping(demo8co10[10.3.162.146:22]) ... ping(demo8co11[10.3.162.140:22]) ... ping(demo8co12[10.3.162.139:22]) ... ping(demo8co13[10.3.162.134:22]) ... ping(demo8co14[10.3.162.144:22]) ... ping(demo8co15[10.3.162.145:22]) ... ping(demo8co2[10.3.162.178:22]) ... ping(demo8co3[10.3.162.142:22]) ... ping(demo8co4[10.3.162.149:22]) ... ping(demo8co5[10.3.162.137:22]) ... ping(demo8co6[10.3.162.138:22]) ... ping(demo8co7[10.3.162.153:22]) ... ping(demo8co8[10.3.162.154:22]) ... ping(demo8co9[10.3.162.136:22]) ... ping(hlmvm[10.3.162.30:22]) ... ping(kvm[10.3.162.10:22]) ... ping(wrcl0[10.3.162.131:22]) ... ping(wrcl1[10.3.162.132:22]) ... 
-------- Checking platform <poc1>
ping(wrcllr0[10.3.30.21]) ... ping(wrcllr1[10.3.30.22]) ... 8 flavors, 25 images, 10 servers
ping(hlm1[10.3.28.46:22]) ... ping(hlm1vm[10.3.29.10:22]) ... ping(wrcllr0[10.3.30.21:22]) ... 

In [4]:
display( HTML( '<h1><a name="RESULTS_STATUS" /> Platforms Status Summary </h1>' ))

DISP_STATUSES={}
DISP_STATUSES['--Platform']=[ 'Worst case Disk usage', 'Ping', 'Ports', 'Endpoints', 'VMs' ]
for platform in platforms:
    LINK='<a href="#RESULTS_' + platform + '">' + platform +'</a>'
    DISP_STATUSES[LINK]=STATUSES[platform]

display( HTML( DictTable._repr_html_(DISP_STATUSES, STATUS_HIGHLIGHTS) ))

display( HTML( '<h1>Platforms results </h1>' ))
for platform in platforms:
    display( HTML( '<a name="RESULTS_{}">_</a>'.format(platform) ))
    HTML_OP = HTML_OPS[platform]
    display( HTML( HTML_OP )) 

hlm1[10.3.216.10],OK: 0.157 msec
hlm1vm[10.3.219.30],OK: 0.318 msec
hos1[10.3.219.31],OK: 0.222 msec
hos2[10.3.219.32],OK: 0.201 msec
hos3[10.3.219.33],OK: 0.211 msec
hos_vip[10.3.216.30],OK: 0.313 msec
wrcl1[10.3.219.131],OK: 0.181 msec
wrcl2[10.3.219.132],OK: 0.136 msec
wrcl_vip[10.3.219.130],OK: 0.141 msec
wrco0[10.3.219.177],OK: 0.165 msec
wrco1[10.3.219.178],OK: 0.107 msec


nfv5co10[10.206.61.154],OK: 0.395 msec
nfv5co11[10.206.61.162],OK: 0.407 msec
nfv5co12[10.206.61.171],OK: 0.431 msec
nfv5co13[10.206.61.225],OK: 0.381 msec
nfv5co14[10.206.61.155],OK: 0.390 msec
nfv5co15[10.206.61.246],OK: 0.539 msec
nfv5co16[10.206.61.193],OK: 0.437 msec
nfv5co1[10.206.61.215],OK: 0.367 msec
nfv5co2[10.206.61.161],TIMEOUT
nfv5co3[10.206.61.170],OK: 0.392 msec
nfv5co4[10.206.61.172],OK: 0.389 msec


### TODO:
- use of openstack clientmanager for equivalent to "openstack service list" status

In [5]:
# TODO: 
# equivalent of openstack service list

# See: how service list is implemented:
#    https://github.com/openstack/python-openstackclient/blob/master/openstackclient/compute/v2/service.py
# (compute_client = client_manager.compute)
#
# See also: how to create/use a clientmanager:
#    https://github.com/openstack/python-openstackclient/blob/master/examples/osc-lib.py

#from openstackclient.common import clientmanager
#from openstackclient.common import utils